In [2]:
!pip install pandas -q
import numpy as np
import pandas as pd

# datasets
url_listings = 'https://data.insideairbnb.com/united-states/co/denver/2024-09-28/data/listings.csv.gz'
# url_calendar = 'https://data.insideairbnb.com/united-states/co/denver/2024-09-28/data/calendar.csv.gz'
# url_reviews = 'https://data.insideairbnb.com/united-states/co/denver/2024-09-28/data/reviews.csv.gz'
# url_neighbourhoods = 'https://data.insideairbnb.com/united-states/co/denver/2024-09-28/visualisations/neighbourhoods.csv'

# load datasets into DataFrames
listings = pd.read_csv(url_listings, compression='gzip')
# calendar = pd.read_csv(url_calendar, compression='gzip')
# reviews = pd.read_csv(url_reviews, compression='gzip')
# neighbourhoods = pd.read_csv(url_neighbourhoods)

September 28, 2024. Data for the 3rd quarter of the 2024.

In [4]:
# remove features that are not needed (first impression)
columns_to_remove = [
    'id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'picture_url',
    'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 
    'host_about', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 
    'host_listings_count', 'host_total_listings_count', 'host_verifications', 
    'bathrooms_text', 'calendar_updated', 
    'calendar_last_scraped', 'first_review', 'last_review', 'license', 
    'calculated_host_listings_count', 'name', 'description', 
    'neighborhood_overview', 'calculated_host_listings_count_entire_homes', 
    'calculated_host_listings_count_private_rooms', 
    'calculated_host_listings_count_shared_rooms', 
    'minimum_minimum_nights', 'maximum_minimum_nights', 
    'minimum_maximum_nights', 'maximum_maximum_nights', 
    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability', 
    'availability_30', 'availability_60', 'availability_90', 'availability_365', 
    'number_of_reviews_ltm', 'number_of_reviews_l30d', 'reviews_per_month',
    'review_scores_accuracy', 'review_scores_cleanliness',
    'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 
    'review_scores_value', 'neighbourhood', 'neighbourhood_group_cleansed'
    
]

data = listings.drop(columns=columns_to_remove)
print(data['property_type'].value_counts())
#print(data.columns)

#data.info()

property_type
Entire home                           1644
Entire rental unit                     966
Entire condo                           503
Entire guest suite                     429
Private room in home                   425
Entire townhouse                       398
Entire guesthouse                      291
Private room in townhouse               84
Entire loft                             78
Entire bungalow                         53
Private room in rental unit             53
Private room in bed and breakfast       28
Private room in guest suite             20
Room in aparthotel                      20
Room in hotel                           19
Private room in condo                   16
Shared room in hostel                   15
Shared room in home                     14
Entire serviced apartment               13
Private room in bungalow                12
Tiny home                               11
Entire cottage                          10
Entire place                            

Features in each dataset:

Listings: ['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'number_of_reviews_ltm', 'license']

Calendar: ['listing_id', 'date', 'available', 'price', 'adjusted_price', 'minimum_nights', 'maximum_nights']

Reviews: ['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments']

Neighbourhoods: ['neighbourhood_group', 'neighbourhood']

In [6]:
# Clean and convert price
data = data.dropna(subset=['price'])  # drop rows missing 'price'
data['price'] = (
    data['price']
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(float)
)

# Convert % columns to numeric
for col in ['host_response_rate', 'host_acceptance_rate']:
    # Remove trailing '%'
    data[col] = data[col].str.replace('%', '', regex=False)
    # Convert to numeric (non-numeric becomes NaN)
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Binary columns: map 't'/'f' to 1/0
binary_cols = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']
for col in binary_cols:
    data[col] = data[col].map({'t': 1, 'f': 0})

# ------------------------------------------------------------------
# **MISSING-DATA STRATEGIES**
# ------------------------------------------------------------------

# A) Handle review_scores_rating:
#    - Create 'no_reviews' flag = 1 if missing, else 0
#    - Fill missing 'review_scores_rating' with 0
data['no_reviews'] = data['review_scores_rating'].isnull().astype(int)
data['review_scores_rating'] = data['review_scores_rating'].fillna(0)

# B) Handle host_response_time, host_response_rate, host_acceptance_rate:
#    1) Replace "N/A" with np.nan (so we can detect real missingness)
data['host_response_time'] = data['host_response_time'].replace('N/A', np.nan)
for col in ['host_response_rate', 'host_acceptance_rate']:
    data[col] = data[col].replace('N/A', np.nan)

#    2) Create a missingness flag for host_response_time
data['host_response_time_missing'] = data['host_response_time'].isnull().astype(int)

#    3) For numeric rates, fill missing with 0
for col in ['host_response_rate', 'host_acceptance_rate']:
    data[col] = data[col].fillna(0)

#    4) If you still want to keep host_response_time as ordinal,
#       map known categories to integers, leaving NaN for missing
response_time_order = {
    "within an hour": 1,
    "within a few hours": 2,
    "within a day": 3,
    "a few days or more": 4
}
data['host_response_time'] = data['host_response_time'].map(response_time_order)

# ------------------------------------------------------------------
# End of missing-data handling
# ------------------------------------------------------------------

# threshold-based grouping
threshold = 25
counts = data['property_type'].value_counts()
rare_property_types = counts[counts < threshold].index

data['property_type'] = data['property_type'].apply(
    lambda x: x if x not in rare_property_types else 'Other'
)

# One-hot encode categorical variables
categorical_cols = ['property_type', 'room_type', 'neighbourhood_cleansed']
data = pd.get_dummies(data, columns=categorical_cols, dummy_na=True)

# Create a simple feature from amenities (count them)
data['amenities_count'] = data['amenities'].apply(
    lambda x: len(eval(x)) if pd.notnull(x) else 0
)
# ----------------------------
# host_response_time
# ----------------------------
data['host_response_time'] = data['host_response_time'].fillna(5)

# ----------------------------
# host_is_superhost
# ----------------------------
data['host_is_superhost'] = data['host_is_superhost'].fillna(0)

# ----------------------------
# beds & bedrooms
# ----------------------------
data['beds'] = data['beds'].fillna(data['beds'].median())
data['bedrooms'] = data['bedrooms'].fillna(data['bedrooms'].median())

# Optional: verify no more missing
print(data.isnull().sum().sort_values(ascending=False).head(10))



host_response_time                            0
neighbourhood_cleansed_Highland               0
neighbourhood_cleansed_Northeast Park Hill    0
neighbourhood_cleansed_North Park Hill        0
neighbourhood_cleansed_North Capitol Hill     0
neighbourhood_cleansed_Montclair              0
neighbourhood_cleansed_Montbello              0
neighbourhood_cleansed_Marston                0
neighbourhood_cleansed_Mar Lee                0
neighbourhood_cleansed_Lowry Field            0
dtype: int64


In [7]:
'''
null_perc = data.isnull().mean().sort_values(ascending=False)
pd.set_option('display.max_rows', len(null_perc))
print(null_perc)
pd.reset_option('display.max_rows')
'''

"\nnull_perc = data.isnull().mean().sort_values(ascending=False)\npd.set_option('display.max_rows', len(null_perc))\nprint(null_perc)\npd.reset_option('display.max_rows')\n"

In [12]:
!pip install pandas numpy scikit-learn requests -q

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# ----------------------------
# 11. DEFINE FEATURES (X) AND TARGET (y)
# ----------------------------
# The target is 'price'
y = data['price']
X = data.drop(columns=['price'])

# ----------------------------
# 12. SPLIT INTO TRAIN & TEST SETS
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ----------------------------
# 13. TRAIN A PRICE PREDICTION MODEL
#     (RANDOM FOREST REGRESSOR)
# ----------------------------
model = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

# ----------------------------
# 14. PREDICT & EVALUATE
# ----------------------------
y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)  # sqrt of MSE
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Random Forest Regressor Performance:")
print(f"  RMSE: {rmse:.2f}")
print(f"  MAE:  {mae:.2f}")
print(f"  R^2:  {r2:.2f}")

ValueError: could not convert string to float: '["Shampoo", "Paid washer \\u2013 In building", "Cleaning products", "Window AC unit", "Iron", "Toaster", "Wine glasses", "Long term stays allowed", "Keypad", "Smoke alarm", "Hangers", "Central heating", "Stove", "Coffee maker: drip coffee maker, french press", "Self check-in", "Ceiling fan", "Shower gel", "Fire extinguisher", "Outdoor dining area", "TV", "Bathtub", "Hair dryer", "Coffee", "Essentials", "Books and reading material", "Dedicated workspace", "Kitchen", "Microwave", "Cooking basics", "Ethernet connection", "Backyard", "Paid dryer \\u2013 In building", "Outdoor furniture", "Oven", "Pets allowed", "Dishes and silverware", "Freezer", "Hot water kettle", "Body soap", "Dining table", "Free street parking", "Bed linens", "Laundromat nearby", "Refrigerator", "Luggage dropoff allowed", "Private entrance", "Wifi", "Carbon monoxide alarm", "Hot water", "Patio or balcony", "Clothing storage", "Portable fans", "Baking sheet"]'

In [16]:
!pip install pandas numpy scikit-learn requests -q

import pandas as pd
import numpy as np
import ast
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

###############################################################################
# 1) LOAD THE DATA
###############################################################################
url_listings = "https://data.insideairbnb.com/united-states/co/denver/2024-09-28/data/listings.csv.gz"
listings = pd.read_csv(url_listings, compression='gzip')

###############################################################################
# 2) DROP COLUMNS THAT AREN'T NEEDED
###############################################################################
columns_to_remove = [
    'id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'picture_url',
    'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 
    'host_about', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 
    'host_listings_count', 'host_total_listings_count', 'host_verifications', 
    'bathrooms_text', 'calendar_updated', 'calendar_last_scraped', 'first_review', 
    'last_review', 'license', 'calculated_host_listings_count', 'name', 
    'description', 'neighborhood_overview', 'calculated_host_listings_count_entire_homes', 
    'calculated_host_listings_count_private_rooms', 
    'calculated_host_listings_count_shared_rooms', 
    'minimum_minimum_nights', 'maximum_minimum_nights', 
    'minimum_maximum_nights', 'maximum_maximum_nights', 
    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability', 
    'availability_30', 'availability_60', 'availability_90', 'availability_365', 
    'number_of_reviews_ltm', 'number_of_reviews_l30d', 'reviews_per_month',
    'review_scores_accuracy', 'review_scores_cleanliness',
    'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 
    'review_scores_value', 'neighbourhood', 'neighbourhood_group_cleansed'
]

data = listings.drop(columns=columns_to_remove, errors='ignore')

###############################################################################
# 3) DROP ROWS WITHOUT PRICE & CONVERT PRICE -> FLOAT
###############################################################################
data.dropna(subset=['price'], inplace=True)

data['price'] = (
    data['price']
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(float)
)

###############################################################################
# 4) OPTIONAL: REMOVE EXTREME OUTLIERS (TOP 1% OF PRICE)
###############################################################################
upper_cap = data['price'].quantile(0.99)
data = data[data['price'] < upper_cap]

###############################################################################
# 5) LOG-TRANSFORM THE TARGET (PRICE)
###############################################################################
# We'll replace 'price' in the DataFrame with its log-transform.
# That way, we'll predict log_price, and then invert predictions later.
data['log_price'] = np.log1p(data['price'])

###############################################################################
# 6) CONVERT PERCENTAGE COLUMNS TO NUMERIC
###############################################################################
for col in ['host_response_rate', 'host_acceptance_rate']:
    data[col] = data[col].str.replace('%', '', regex=False)
    data[col] = pd.to_numeric(data[col], errors='coerce')

###############################################################################
# 7) CONVERT BINARY COLUMNS ('t'/'f' -> 1/0)
###############################################################################
binary_cols = ['host_is_superhost', 'host_has_profile_pic', 
               'host_identity_verified', 'instant_bookable']
for col in binary_cols:
    if col in data.columns:
        data[col] = data[col].map({'t': 1, 'f': 0})

###############################################################################
# 8) HANDLE MISSING DATA
###############################################################################

# --- A) REVIEW SCORES ---
data['no_reviews'] = data['review_scores_rating'].isnull().astype(int)
data['review_scores_rating'] = data['review_scores_rating'].fillna(0)

# --- B) HOST RESPONSE TIME ---
data['host_response_time'] = data['host_response_time'].replace('N/A', np.nan)
data['host_response_time_missing'] = data['host_response_time'].isnull().astype(int)

# Fill numeric rates with 0 if missing
for col in ['host_response_rate', 'host_acceptance_rate']:
    data[col] = data[col].replace('N/A', np.nan).fillna(0)

# Map response time to numeric
response_time_order = {
    "within an hour": 1,
    "within a few hours": 2,
    "within a day": 3,
    "a few days or more": 4
}
data['host_response_time'] = data['host_response_time'].map(response_time_order)
data['host_response_time'] = data['host_response_time'].fillna(5)  # A separate 'unknown' code

# --- C) FILL BEDS & BEDROOMS ---
data['beds'] = data['beds'].fillna(data['beds'].median())
data['bedrooms'] = data['bedrooms'].fillna(data['bedrooms'].median())

# --- D) FILL BINARY HOST_IS_SUPERHOST IF MISSING ---
if 'host_is_superhost' in data.columns:
    data['host_is_superhost'] = data['host_is_superhost'].fillna(0)

###############################################################################
# 9) GROUP RARE PROPERTY TYPES
###############################################################################
if 'property_type' in data.columns:
    threshold = 25
    counts = data['property_type'].value_counts()
    rare_property_types = counts[counts < threshold].index
    data['property_type'] = data['property_type'].apply(
        lambda x: x if x not in rare_property_types else 'Other'
    )

###############################################################################
# 10) ONE-HOT ENCODE CATEGORICAL COLUMNS
###############################################################################
categorical_cols = []
for col in ['property_type', 'room_type', 'neighbourhood_cleansed']:
    if col in data.columns:
        categorical_cols.append(col)

data = pd.get_dummies(data, columns=categorical_cols, dummy_na=True)

###############################################################################
# 11) CREATE AMENITIES COUNT FEATURE
###############################################################################
if 'amenities' in data.columns:
    def count_amenities(amenities_str):
        try:
            # Use ast.literal_eval for safer evaluation of the JSON-like list
            am_list = ast.literal_eval(amenities_str)
            return len(am_list)
        except:
            return 0

    data['amenities_count'] = data['amenities'].apply(
        lambda x: count_amenities(x) if pd.notnull(x) else 0
    )

    data.drop(columns=['amenities'], inplace=True, errors='ignore')

###############################################################################
# 12) OPTIONAL: ADD LOCATION FEATURES IF AVAILABLE
###############################################################################
# If your dataset has 'latitude' and 'longitude', you can keep them.
# E.g.:
# data['lat'] = listings['latitude']
# data['lon'] = listings['longitude']
# # Or compute distance from downtown Denver using a haversine formula.

# For now, we’ll just keep them if they exist:
for col in ['latitude', 'longitude']:
    if col in listings.columns:
        data[col] = listings[col]

###############################################################################
# 13) DEFINE FEATURES (X) AND TARGET (y)
###############################################################################
# We'll predict "log_price" rather than raw "price".
y = data['log_price']
X = data.drop(columns=['price', 'log_price'], errors='ignore')

# Restrict X to numeric columns only, to avoid conversion errors:
X = X.select_dtypes(include=[np.number])

###############################################################################
# 14) TRAIN-TEST SPLIT
###############################################################################
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

###############################################################################
# 15) TRAIN A RANDOM FOREST REGRESSOR
###############################################################################
model = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

###############################################################################
# 16) PREDICT & EVALUATE (CONVERT LOG-PRICE BACK)
###############################################################################
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)      # from log price back to real price
y_test_exp = np.expm1(y_test)      # same for actual test data

rmse = mean_squared_error(y_test_exp, y_pred, squared=False)
mae = mean_absolute_error(y_test_exp, y_pred)
r2 = r2_score(y_test_exp, y_pred)

print("Random Forest Regressor (Log-Transformed) Performance:")
print(f"  RMSE: {rmse:.2f}")
print(f"  MAE:  {mae:.2f}")
print(f"  R^2:  {r2:.2f}")


Random Forest Regressor (Log-Transformed) Performance:
  RMSE: 66.54
  MAE:  41.01
  R^2:  0.61


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
